## Análise Espacial - MBA em DSA - USP-ESALQ - Prof. Dr. Rafael de Freitas Souza

## Aula 2 - Simple Feature e Spatial Points

## Importando as bibliotecas

In [ ]:
pip install rdata

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
#import fiona
#import numpy as np
import pyreadr
#import contextily as cx

In [ ]:
#No notebook sobre shapefiles, usamos a library pyreadr para ler um RData
shoppings = pyreadr.read_r('shoppings.RData')
shoppings = shoppings['shoppings']
shoppings.tail()

### Veja que com a biblioteca pyreadr tivemos problemas com o encoding, já que ela não permite a definição do mesmo na hora da importação, portanto, vamos usar uma outra biblioteca para importar arquivos RData para o python: a rdata

In [ ]:
import rdata

In [ ]:
parsed = rdata.parser.parse_file("shoppings.RData")
converted = rdata.conversion.convert(parsed, default_encoding="utf8")
shoppings = pd.DataFrame(converted['shoppings'])

In [ ]:
shoppings.tail()

## Criando um sf a partir de um dataframe
https://geopandas.org/en/latest/gallery/create_geopandas_from_pandas.html

In [ ]:
sf_shopping = gpd.GeoDataFrame(
    shoppings, geometry=gpd.points_from_xy(shoppings.longitude, shoppings.latitude, crs=4326))

In [ ]:
sf_shopping.head()

In [ ]:
sf_shopping.plot(figsize=(10,10))

In [ ]:
#Extraindo o Bounding Box do dataset (xmin, ymin, xmax e ymax)
sf_shopping.total_bounds

In [ ]:
#Extraindo o CRS do dataset
sf_shopping.crs

# Combinando um objeto simple feature com um shapefile

## Sobrepondo mapas (layers)


In [ ]:
shp_saopaulo = gpd.read_file('shapefiles/municipio_sp.shp')

In [ ]:
shp_saopaulo.head()

In [ ]:
shp_saopaulo.crs

In [ ]:
shp_saopaulo.plot(figsize=(5,5), color='white', edgecolor='black')

In [ ]:
#Como a geometria do dataset dos shoppings está em graus, é necessário alterar o CRS do shapefile da cidade de SP
shp_saopaulo = shp_saopaulo.to_crs(4326)

In [ ]:
#https://geopandas.org/en/latest/docs/reference/api/geopandas.GeoDataFrame.plot.html
base = shp_saopaulo.plot(color='white', edgecolor='black', figsize = (12,12))
sf_shopping.plot(column = 'regiao', ax=base, marker='o', markersize=50, legend = True);

In [ ]:
sf_shopping.explore(legend = False, marker_kwds = {'radius': 10, 'fill': True, 'color': 'green'})

# Buffer Analysis
### O buffering é uma técnica para se medir distâncias para fora de um dado ponto geográfico.
### Referência: https://geopandas.org/en/stable/getting_started/introduction.html?highlight=buffer

In [ ]:
#Como a geometria estava em graus, foi necessário alterar o CRS
sf_shopping = sf_shopping.to_crs(22523) #Mesmo CRS usado pelo professor para considerar a distância Euclidiana

In [ ]:
sf_shopping.crs

### Diferentemente do R, no Python não é necessário vc transformar um SF em SP para o buffering.

In [ ]:
#Aqui é criado um novo feature (coluna) no dataset com os valores do buffering
sf_shopping["buffered"] = sf_shopping.buffer(1500) #usamos a distância dde 1500 metros para o buffering

In [ ]:
sf_shopping.head()

In [ ]:
ax = sf_shopping["buffered"].plot(color='none', edgecolor='black', figsize = (5,5))

In [ ]:
sf_shopping['buffered'].explore(color = 'black', style_kwds = {'fill': False, 'weight': 1})

## Combinando os objetos shp_saopaulo, sf_shoppings e buffer_shoppings

In [ ]:
#Primeiro, vamos ter certeza que o CRS dos dois é o mesmo
shp_saopaulo = shp_saopaulo.to_crs(4326)
sf_shopping = sf_shopping.to_crs(4326)

In [ ]:
#Veja que o referencial da coluna buffer criada é em metores, enquanto a geometria está em long/lat.
sf_shopping.head()

In [ ]:
#Aqui estamos alteranddo o CRS da coluna buffered para o mesmo que vamos usar na plotagem
sf_shopping['buffered'] = sf_shopping['buffered'].to_crs(4326)

In [ ]:
ax = sf_shopping['buffered'].plot(alpha = .1, edgecolor='black', figsize = (10,10))
sf_shopping.plot(column = 'regiao', ax=ax, marker='o', markersize=50, legend = True);
shp_saopaulo.plot(ax = ax, color='white', edgecolor='gray', figsize = (12,12),  zorder = 0)
title = "Shoppings na cidade de São Paulo"
plt.title(title, fontsize=21)